In [1]:
import sys
sys.path.insert(0, '..')

import os
import requests
import json

In [2]:
from biosimulator_processes.tests.test_copasi_process import test_process_from_document
from biosimulator_processes.tests.data_model import ProcessUnitTest
from biosimulator_processes import CORE
from process_bigraph import Composite, pp

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.


#### Step 1: Define the document to be read by the Composite, which is implemented by the Process.

In [3]:
biomodel_id = 'BIOMD0000000630'

document = {
    'copasi': {
        '_type': 'process',
        'address': 'local:copasi',
        'config': {
            'model': {
                'model_source': biomodel_id  
            },
            'method': 'lsoda'
        },
        'inputs': {
            'floating_species_concentrations': ['floating_species_concentrations_store'],
            'floating_species_counts': ['floating_species_counts_store'],
            'model_parameters': ['model_parameters_store'],
            'time': ['time_store'],
            'reactions': ['reactions_store']
        },
        'outputs': {
            'floating_species_concentrations': ['floating_species_concentrations_store'],
            'floating_species_counts': ['floating_species_counts_store'],
            'time': ['time_store'],
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'floating_species_concentrations': 'tree[float]',
                'floating_species_counts': 'tree[float]',
                'time': 'float',
            },
        },
        'inputs': {
            'floating_species_concentrations': ['floating_species_concentrations_store'],
            'floating_species_counts': ['floating_species_counts_store'],
            'time': ['time_store'],
        }
    }
}

#### Step 2: Define the instance composition along with the process registry

In [4]:
workflow = Composite(
    config={'state': document},
    core=CORE
)

found a biomodel id


#### Step 3: Run the workflow for a duration and get the results

In [5]:
workflow.run(30)

results = workflow.gather_results()

/Users/alex/Desktop/uchc_work/repos/biosimulator-processes/notebooks/../biosimulator_processes/processes/copasi_process.py:245: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/Users/alex/Desktop/uchc_work/repos/biosimulator-processes/notebooks/../biosimulator_processes/processes/copasi_process.py:255: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [6]:
pp(results)

{ ('emitter',): [ { 'floating_species_concentrations': { 'plasmin': 0.0,
                                                         'plasminogen': 0.0,
                                                         'single intact chain urokinase-type plasminogen activator': 0.0,
                                                         'two-chain urokinase-type plasminogen activator': 0.0,
                                                         'x': 0.0,
                                                         'x-plasmin': 0.0},
                    'floating_species_counts': { 'plasmin': 0.0,
                                                 'plasminogen': 0.0,
                                                 'single intact chain urokinase-type plasminogen activator': 0.0,
                                                 'two-chain urokinase-type plasminogen activator': 0.0,
                                                 'x': 0.0,
                                                 'x-plasmin': 

In [31]:
workflow.config['state']['emitter']

{'_type': 'step',
 'address': 'local:ram-emitter',
 'config': {'emit': {'floating_species_concentrations': 'tree[float]',
   'floating_species_counts': 'tree[float]',
   'time': 'float'}},
 'inputs': {'floating_species_concentrations': ['floating_species_concentrations_store'],
  'floating_species_counts': ['floating_species_counts_store'],
  'time': ['time_store']}}

In [48]:
from biosimulator_processes.steps.viz import parse_composition_results


results = parse_composition_results(workflow)

pp(results), list(results.keys())

{ 0.0: { 'floating_species_concentrations': { 'plasmin': 0.0,
                                              'plasminogen': 0.0,
                                              'single intact chain urokinase-type plasminogen activator': 0.0,
                                              'two-chain urokinase-type plasminogen activator': 0.0,
                                              'x': 0.0,
                                              'x-plasmin': 0.0},
         'floating_species_counts': { 'plasmin': 0.0,
                                      'plasminogen': 0.0,
                                      'single intact chain urokinase-type plasminogen activator': 0.0,
                                      'two-chain urokinase-type plasminogen activator': 0.0,
                                      'x': 0.0,
                                      'x-plasmin': 0.0},
         'time': 0.0},
  1.0: { 'floating_species_concentrations': { 'plasmin': 3.500808585798738e-07,
                       

In [47]:
y_data = []
times = list(results.keys())
index = 'floating_species_concentrations'


    

for timestamp, result in results.items():
    
    root_data = result[index]
    names = list(root_data.keys())
    for name in names:
        y_data.append(results[timestamp][index][name])
        print(f'Got data for name {name}: {y_data}')
        y_data.clear()
        
        
        

Got data for name plasminogen: [0.0]
Got data for name plasmin: [0.0]
Got data for name single intact chain urokinase-type plasminogen activator: [0.0]
Got data for name two-chain urokinase-type plasminogen activator: [0.0]
Got data for name x: [0.0]
Got data for name x-plasmin: [0.0]
Got data for name plasminogen: [0.00984133843876594]
Got data for name plasmin: [3.500808585798738e-07]
Got data for name single intact chain urokinase-type plasminogen activator: [0.003069285481826758]
Got data for name two-chain urokinase-type plasminogen activator: [2.18396863350984e-15]
Got data for name x: [0.0]
Got data for name x-plasmin: [0.0]
Got data for name plasminogen: [0.029212079654514916]
Got data for name plasmin: [2.9773237189981014e-06]
Got data for name single intact chain urokinase-type plasminogen activator: [0.008960567969375093]
Got data for name two-chain urokinase-type plasminogen activator: [4.168569306038924e-13]
Got data for name x: [0.0]
Got data for name x-plasmin: [0.0]
Got

In [9]:
"workflow.state['global_time']

30.0

In [10]:
from biosimulator_processes.steps.viz import ResultsAnimation, Plotter2d

In [11]:
output = results.copy()

In [12]:
output_vals = output[('emitter',)]

KeyError: ('emitter',)

In [ ]:
timescale = list(set([val.get('time', 0.0) for val in output_vals]))

In [ ]:
timescale

In [ ]:
data = []
counts_data = []

In [ ]:
for i, val in enumerate(output_vals):
    species_data = val.get('floating_species_concentrations')
    data.append(species_data.get('plasminogen'))
    counts = val.get('floating_species_counts')
    counts_data.append(counts.get('plasminogen'))

In [ ]:
len(timescale), len(data)

In [ ]:
Plotter2d.plot_single_output(timescale=timescale, data=data, species_name='plasminogen concentration')

In [ ]:
Plotter2d.plot_single_output(timescale=timescale, data=counts_data, species_name='plasminogen counts', plot_concentration=False)

In [ ]:
Plotter2d.plot_output(x_data=data, y_data=counts_data, title='Plasminogen concentration over counts', x_label='concentration', y_label='counts', species='plasminogen')